In [1]:
import os
import time
import gensim
import random
import numpy as np
import smart_open
import collections
import pickle

# Create Train/Testing corpus

In [2]:
def read_corpus(data, tokens_only=False):
    for i, line in enumerate(data):
        if tokens_only:
            yield gensim.utils.simple_preprocess(line)
        else:
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [3]:
data = pickle.load(open("processed_comments.pkl","rb"))
# train = data[:-10000]
# test = data[-10000:]
train = data

In [4]:
train_corpus = list(read_corpus(train))

In [5]:
# train_corpus

# Train Model

## Doc2vec object created

In [5]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40)

## Build Vocabulary

In [6]:
model.build_vocab(train_corpus)

In [11]:
# model.build_vocab_from_freq(table)

## Train

In [7]:
model.train(train_corpus, total_examples=model.corpus_count,epochs=model.epochs)

In [9]:
pickle.dump(model,open("gensim_model.pkl","wb"))

# Get inference out of it

In [10]:
model.infer_vector(["how","you","doing","?"])

array([ 0.0038966 , -0.12413467, -0.00088628,  0.04547637,  0.02487027,
        0.2914945 , -0.1413551 ,  0.15388733,  0.11048101,  0.08213618,
        0.06864414,  0.0151264 , -0.06750699, -0.07717273,  0.20570303,
       -0.11310355,  0.12889908,  0.11036842,  0.03827285, -0.09396256,
        0.1627588 , -0.06848573, -0.16042966, -0.00045034, -0.03100291,
        0.26734978,  0.01682843, -0.01205921,  0.06844009, -0.02830203,
        0.35193312,  0.13214266,  0.05767069,  0.31941187, -0.23206793,
        0.06590673, -0.25150767,  0.0613074 ,  0.28405052, -0.02775035,
        0.09447619,  0.04961845, -0.17056474,  0.05519173, -0.13871287,
       -0.07159866,  0.01847048,  0.14687994,  0.10511759, -0.00843077,
        0.0471995 ,  0.0183151 ,  0.07935549,  0.06730979,  0.09077326,
        0.06307517, -0.03231017, -0.10422688,  0.07281328, -0.10892303,
        0.23522893, -0.29230562,  0.01566891,  0.1784539 , -0.25520617,
        0.15966111, -0.01766184, -0.01766215,  0.3377682 ,  0.03

# Assessing the model

### TODO

In [11]:
a = model.infer_vector("This is a fix for lower volume limits on m5 and c5 instance types while we wait for kubernetes/enhancements#554 to land GA.This problem became urgent because many of our users are trying to migrate to those instance types in light of spectre/meltdown vulnerability but lower volume limit on those instance types often causes cluster instability. Yes they can workaround by configuring the scheduler with lower limit but often this becomes somewhat difficult to do when cluster is mixed.The newer default limits were picked from Text about spectre/meltdown is available on")

In [ ]:
ranks = []
second_ranks = []
for doc_id in range(len(data)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [15]:
word = "vulnerability"
for i,doc in enumerate(data):
    if word in doc:
        print(i)

307
1246
1625
4183
5181
6073
18304
26104
31265
31694
32617
40582
43692
43887
46364
48336
55190
59903
69029
71831
73907
84102
84990
87091
87644
88939
90324
93190
95292
97248
101131
110105
113508
117323
117324
117331
117379
117386
117395
117413
117419
117929
120032
120475
120496
120497
120532
120570
120592
120622
120640
125384
125967
126367
126493
126500
126855
135296
139407
139478
139497
141205
143989
144708
146679
146700
147935
148232
148316
148869
149727
150573
160151
162377


In [58]:
doc_id = 162377

In [65]:
print('Document {}: «{}»\n'.format("", ' '.join(train_corpus[doc_id].words)))
a = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([a], topn=len(model.docvecs))
rank = [docid for docid, sim in sims].index(doc_id)
second_rank = sims[1]
third_rank = sims[2]

print(second_rank,third_rank)

# Compare and print the second-most-similar document
print('Similar Document {}: «{}»\n'.format(second_rank, ' '.join(train_corpus[second_rank[0]].words)))

print('Similar Document {}: «{}»\n'.format(third_rank, ' '.join(train_corpus[third_rank[0]].words)))

Document : «cve dashboard vulnerable to dns rebinding attack minikube exposes the service with configuration that makes it vulnerable to attacks thanks to alex kaskasoli for reporting this problem vulnerable versions minikube vulnerable configurations vm environments which use predictable ip address such as virtualbox or vulnerability impact if an attacker gets victim to visit malicious web page the attacker may be able to execute arbitrary code within the victim minikube cluster minikube exposes the kubernetes dashboard listening on the vm ip at port in vm environments where the ip is easy to predict the attacker can use to indirectly make requests to the kubernetes dashboard without violating the same origin policy the attacker can generate csrf token from the endpoint and pass this token to the endpoint to create new kubernetes deployment running payload of their choosing this vulnerability can be combined with vm specific vulnerability to escape to the host operating system if is i

In [71]:
doc_id = 148869
a = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([a], topn=len(model.docvecs))
print('Document {}: «{}»\n'.format("", ' '.join(train_corpus[doc_id].words)))
rank = [docid for docid, sim in sims].index(doc_id)
second_rank = sims[1]
third_rank = sims[2]
forth_rank = sims[3]
print('Similar Document {}: «{}»\n'.format(second_rank, ' '.join(train_corpus[second_rank[0]].words)))
print('Similar Document {}: «{}»\n'.format(third_rank, ' '.join(train_corpus[third_rank[0]].words)))
print('Similar Document {}: «{}»\n'.format(third_rank, ' '.join(train_corpus[forth_rank[0]].words)))

Document : «how to use the public dataset paddlecloud provides some public dataset for the developer how to usage we can install python package in the runtime docker image and use it as how to block the data leakage because of developers can upload trainer python package to the paddlecloud so think the most effective way to block the data leakage is blocking all connections of kubernetes nodes to the exteranl internal this is security issue here is several things to consider about can nodes pods access the internet users may need to download dependencies or public data but also being able to upload public datasets to other places by injecting some code in the maybe no internet access from nodes pods is good idea users can save training output models to their own cloud storage space and then download the models this is another possible vulnerability users can inject code to to save the original data directly to the storage space we can avoid this by the following ways validate the user 

In [72]:
doc_id = 144708
a = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([a], topn=len(model.docvecs))
print('Document {}: «{}»\n'.format("", ' '.join(train_corpus[doc_id].words)))
rank = [docid for docid, sim in sims].index(doc_id)
second_rank = sims[1]
third_rank = sims[2]
forth_rank = sims[3]
print('Similar Document {}: «{}»\n'.format(second_rank, ' '.join(train_corpus[second_rank[0]].words)))
print('Similar Document {}: «{}»\n'.format(third_rank, ' '.join(train_corpus[third_rank[0]].words)))
print('Similar Document {}: «{}»\n'.format(third_rank, ' '.join(train_corpus[forth_rank[0]].words)))

Document : «pick better random port to listen on in integration tests trying to reduce flakes in ci where we randomly pick port to listen on if we are unluckly its already in use and we fail the test let the os pick free random port to listen on we can set the listener early and the apiserver framework will reuse it ideally the issue is resolved by upstream letting us use the standard convention of using which means the travis build failed with there are patterns that catch this error and keep trying to pick port until the server starts maybe it is now time to add that type of logic yep agree let me give this shot debated about it originally and figured the range was wide enough what the chance ve reworked would you take another look when force it to fail can see it initially fail and then on the nd attempt it always successfully listens on the new random port and the tests are successful think the vulnerability we have is that if it happens to pick random port where another controller

In [73]:
doc_id = 93190
a = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([a], topn=len(model.docvecs))
print('Document {}: «{}»\n'.format("", ' '.join(train_corpus[doc_id].words)))
rank = [docid for docid, sim in sims].index(doc_id)
second_rank = sims[1]
third_rank = sims[2]
forth_rank = sims[3]
print('Similar Document {}: «{}»\n'.format(second_rank, ' '.join(train_corpus[second_rank[0]].words)))
print('Similar Document {}: «{}»\n'.format(third_rank, ' '.join(train_corpus[third_rank[0]].words)))
print('Similar Document {}: «{}»\n'.format(third_rank, ' '.join(train_corpus[forth_rank[0]].words)))

Document : «generic build error just tried to move my project from openshift to openshift online but if try to build it just get check logs for details so check the logs and there isn single error just npm warn entries though my local build works oc kubernetes features basic auth server openshift kubernetes version steps to reproduce just hit build now current result generic build error without error logs expected result successful build through additional information use and have successfully setup webhook triggers and pulling from private gitlap repos works too though my build fails with following non sense log cloning commit bb ca bab ce author andreas deuschlinger date fri oct installing application source building your node application from source npm warn deprecated bower psst your project can stop working at any moment because its dependencies can change prevent this by migrating to yarn npm warn deprecated email templates has been released try it out and view the new docs at it

In [74]:
doc_id = 117323
a = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([a], topn=len(model.docvecs))
print('Document {}: «{}»\n'.format("", ' '.join(train_corpus[doc_id].words)))
rank = [docid for docid, sim in sims].index(doc_id)
second_rank = sims[1]
third_rank = sims[2]
forth_rank = sims[3]
print('Similar Document {}: «{}»\n'.format(second_rank, ' '.join(train_corpus[second_rank[0]].words)))
print('Similar Document {}: «{}»\n'.format(third_rank, ' '.join(train_corpus[third_rank[0]].words)))
print('Similar Document {}: «{}»\n'.format(third_rank, ' '.join(train_corpus[forth_rank[0]].words)))

Document : «clairctl generates empty reports with clair pre yesterday upgraded to clair pre to try out the alpine support but clairctl generates empty reports all my report that had vulnerabilities when running clair now show empty analyze shows some vulnerabilities but the report generated shows nothing thanks for reporting will have look on that le ven vr julien del piccolo notifications crit yesterday upgraded to clair pre to try out the alpine support but clairctl generates empty reports all my report that had vulnerabilities when running clair now show empty analyze shows some vulnerabilities but the report generated shows nothing clairctl analyze registry mydomain net core custom nginx image registry mydomain net registry mydomain net core custom nginx latest layers found analysis found vulnerabilities analysis found vulnerabilities analysis found vulnerabilities analysis found vulnerabilities analysis found vulnerabilities analysis found vulnerabilities analysis found vulnerabil

In [75]:
doc_id = 5181
a = model.infer_vector(train_corpus[doc_id].words)
sims = model.docvecs.most_similar([a], topn=len(model.docvecs))
print('Document {}: «{}»\n'.format("", ' '.join(train_corpus[doc_id].words)))
rank = [docid for docid, sim in sims].index(doc_id)
second_rank = sims[1]
third_rank = sims[2]
forth_rank = sims[3]
print('Similar Document {}: «{}»\n'.format(second_rank, ' '.join(train_corpus[second_rank[0]].words)))
print('Similar Document {}: «{}»\n'.format(third_rank, ' '.join(train_corpus[third_rank[0]].words)))
print('Similar Document {}: «{}»\n'.format(third_rank, ' '.join(train_corpus[forth_rank[0]].words)))

Document : «update kubedns dnsmasq vulnerability cve requires upgrade to patched version was about to make same pr thanks report merging into will coverage the diff coverage is coverage arrow up powered by last update read the»

Similar Document (5181, 0.9168679118156433): «update kubedns dnsmasq vulnerability cve requires upgrade to patched version was about to make same pr thanks report merging into will coverage the diff coverage is coverage arrow up powered by last update read the»

Similar Document (164455, 0.5626943111419678): «make update libcalico»

Similar Document (164455, 0.5626943111419678): «make update libcalico»

